### Import packages

In [6]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import requests
from pandas.io.json import json_normalize # Transform json file into pandas df


In [60]:
from IPython.display import display

### Downloading folium & geopy

In [96]:
#!conda install -c conda-forge geopy --yes

In [98]:
#!conda install -c conda-forge folium=0.9.1 --yes

In [10]:
import folium
from geopy.geocoders import Nominatim

In [14]:
df_tor = pd.read_csv("tor_exp")
df_tor.head(10)

,PostalCode,Borough,Neighborhoods
0,M1A,NaN,NaN
1,M1B,Scarborough,"Rouge, Malvern"
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae
6,M1J,Scarborough,Scarborough Village
7,M1K,Scarborough,"East Birchmount Par, Ionview, Kennedy Park"
8,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
9,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village"


In [15]:
# Backup
#df_geo = pd.read_csv("Geospatial_Coordinates.csv")

In [19]:
with open('nyc.json') as nyc_json:
    nyc_data = json.load(nyc_json)

In [23]:
#nyc_data

In [36]:
nbr_data = nyc_data['features']

In [37]:
nbr_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [32]:
column_names = ['Borough','Neighborhood','Latitude','Longitude']

nbr = pd.DataFrame(columns = column_names)

In [33]:
nbr

,Borough,Neighborhood,Latitude,Longitude


In [81]:
for data in nbr_data:
    borough = nbr_name = data['properties']['borough']
    nbr_name = data['properties']['name']
    nbr_latlon = data['geometry']['coordinates']
    nbr_lat = nbr_latlon[1]
    nbr_lon = nbr_latlon[0]
    nbr = nbr.append({'Borough': borough,
                    'Neighborhood': nbr_name,
                    'Latitude': nbr_lat,
                    'Longitude': nbr_lon}, ignore_index = True)

                                  

In [82]:
nbr.head(10)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


In [83]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent = "ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude 
longitude = location.longitude

print("The geographical coordinate of NYC are {}, {}".format(latitude, longitude))



The geographical coordinate of NYC are 40.7127281, -74.0060152


In [84]:
print(address)
print(location)
print(latitude)
print(longitude)
nbr.columns

New York City, NY
NYC, New York, USA
40.7127281
-74.0060152


Index(['Borough', 'Neighborhood', 'Latitude', 'Longitude'], dtype='object')

In [94]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start = 10)

for lat, lng, borough, neighborhood in zip(nbr['Latitude'], nbr['Longitude'],
                                          nbr['Borough'], nbr['Neighborhood']):
    label = '{},{}'.format(nbr, borough)
    label = folium.Popup(label, parse_html= True)
    folium.CircleMarker([lat, lng], Popup = label, radius = 5, color = 'blue', fill = True, fill_color = 'b', fill_opacity = 0.1,tiles='Stamen Terrain', parse_html = False).add_to(map_newyork)
    
map_newyork
    
    
    
    
    

In [95]:
#map_2 = folium.Map(location=[latitude, longitude],
#zoom_start=12,
#tiles='Stamen Terrain')
#folium.Marker([45.3288, -121.6625]).add_to(map_2)
#folium.Marker([45.3311, -121.7113]).add_to(map_2)
#map_2